# ⚠️ PyKwant Tutorial: Risk & Sensitivity
This notebook introduces `pykwant.risk`, the module dedicated to calculating financial sensitivities.

Unlike the other modules where we composed functions manually, `risk.py` provides a high-level function `calculate_risk_metrics` that automates the calculation of key risk indicators (Duration, Convexity, DV01) by applying numerical bumps to the yield curve.

## 1. Setup and Imports

In [1]:
from datetime import date
from pykwant import risk, instruments, rates, dates

# Helper for clean output
def print_metrics(metrics: dict[str, float]):
    print("-" * 30)
    for k, v in metrics.items():
        print(f"{k.capitalize():<15}: {v:>10.4f}")
    print("-" * 30)

## 2. Setting the Stage
We create a **5-Year Fixed Rate Bond** (4% Coupon) and a **Flat Yield Curve** (4%). Since the coupon matches the market rate, the bond should trade at Par (100.0).

In [2]:
# 1. Define Instrument
bond = instruments.FixedRateBond(
    face_value=instruments.Money(100.0),
    coupon_rate=0.04,
    start_date=date(2025, 1, 1),
    maturity_date=date(2030, 1, 1),
    frequency_months=12,
    day_count=dates.thirty_360
)

# 2. Define Market (Flat 4%)
ref_date = date(2025, 1, 1)

def flat_curve(d: date) -> float:
    t = dates.act_365(ref_date, d)
    # Using the rates module helper
    return rates.compound_factor(-0.04, t, frequency=0)

## 3. Calculating Risk Metrics
We use `risk.calculate_risk_metrics`. This function performs the following steps internally:
1. Prices the instrument at the current curve ($P_0$).
2. Shifts the curve UP by a small bump (e.g., +1bp) to get $P_{up}$.
3. Shifts the curve DOWN by the same bump to get $P_{down}$.
4. Computes **Duration**, **Convexity**, and **DV01** using central differences.

In [3]:
# Calculate metrics with a standard 1bp bump (1e-4)
metrics = risk.calculate_risk_metrics(
    instruments=bond, 
    curve=flat_curve, 
    valuation_date=ref_date,
    bump=1e-4
)

print("--- Risk Report (5Y Bond) ---")
print_metrics(metrics)

# Expected Results for 5Y Par Bond:
# Price:     100.00
# Duration:  ~4.45 (Modified Duration)
# DV01:      ~0.0445 (Price change for 1bp)

--- Risk Report (5Y Bond) ---
------------------------------
Price          :    99.6302
Duration       :     4.6315
Convexity      :    22.4423
Dv01           :     0.0461
------------------------------


## 4. Analysis of Results
Let's interpret the output:
* **Price**: The current clean value of the bond.
* **Duration**: The percentage sensitivity. A duration of 4.45 means if rates rise by 1%, the price drops by $\approx 4.45\%$.
* **Convexity**: The curvature. Positive convexity is good for the bondholder.
* **DV01**: The Dollar Value. It predicts the P&L for a 1 basis point move.

## 5. Stress Testing (Scenario Analysis)
We can verify the accuracy of these metrics by simulating a large market move (e.g., **+1.0%** rate hike) and comparing the predicted loss vs. the actual loss.
**Formula**: $\text{Predicted P\&L} \approx -\text{Duration} \times P \times \Delta y$

In [4]:
# Scenario: Rates rise by 1.0% (100 bps)
shock = 0.01

# 1. Predicted Change
# P&L ~= -Duration * Price * Shock
predicted_delta = -metrics["duration"] * metrics["price"] * shock

# 2. Actual Change
# We manually create a shifted curve (+1%)
def shocked_curve(d: date) -> float:
    t = dates.act_365(ref_date, d)
    return rates.compound_factor(-0.05, t, frequency=0) # 4% -> 5%

price_shocked = instruments.price_instrument(bond, shocked_curve, ref_date)
actual_delta = price_shocked - metrics["price"]

print(f"\n--- Scenario: +100bps Rate Hike ---")
print(f"Predicted P&L: {predicted_delta:.4f}")
print(f"Actual P&L:    {actual_delta:.4f}")
print(f"Difference:    {abs(actual_delta - predicted_delta):.4f}")


--- Scenario: +100bps Rate Hike ---
Predicted P&L: -4.6144
Actual P&L:    -4.5044
Difference:    0.1100
